# Job Sheet 10 : Recurrent Neural Network (RNN)

## Praktikum 2

#### Setup

In [5]:
import tensorflow as tf
import numpy as np
import os
import time

In [6]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [7]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 1115394 characters


In [8]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [9]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

65 unique characters


#### Olah Teks

In [10]:
example_texts = ['abcdefg', 'xyz']
chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [11]:
ids_from_chars = tf.keras.layers.StringLookup(
vocabulary=list(vocab), mask_token=None)

In [12]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>

In [13]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [14]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [15]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [16]:
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

#### Memuat Training Set dan Target

In [17]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1])>

In [18]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [19]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

F
i
r
s
t
 
C
i
t
i


In [20]:
seq_length = 100

In [21]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


In [22]:
for seq in sequences.take(5):
    print(text_from_ids(seq).numpy())

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [23]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [24]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [25]:
dataset = sequences.map(split_input_target)

In [26]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


#### Membuat Batch Training

In [27]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

#### Buat Model

In [28]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [29]:
class MyModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super(MyModel, self).__init__()
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(rnn_units,
                                       return_sequences=True,
                                       return_state=True)
        self.dense = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, states=None, return_state=False, training=False):
        x = self.embedding(inputs)
        if states is None:
            states = self.gru.get_initial_state(batch_size=tf.shape(inputs)[0])
        x, states = self.gru(x, initial_state=states, training=training)
        x = self.dense(x)

        if return_state:
            return x, states
        else:
            return x

In [30]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

#### Uji Model

In [31]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")


(64, 100, 66) # (batch_size, sequence_length, vocab_size)


In [32]:
model.summary()

Model: "my_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (64, 100, 256)         │        16,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ ((64, 100, 1024), (64, │     3,938,304 │
│                                 │ 1024))                 │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (64, 100, 66)          │        67,650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,022,850 (15.35 MB)

 Trainable params: 4,022,850 (15.35 MB)

 Non-trainable params: 0 (0.00 B)

In [33]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [34]:
sampled_indices

array([24, 40, 65, 36, 61, 39, 10, 48,  2, 30,  1, 32, 64, 17, 53, 64, 61,
       62,  2, 42,  5, 61,  6, 48, 28, 32, 36, 36, 14, 47, 29, 50, 37, 60,
        7, 44, 13, 24, 63, 17, 31, 62, 24, 41, 44, 37,  8, 56, 61, 31, 35,
       55, 27, 57, 52, 53, 33,  9, 63, 34, 50, 64,  3, 54, 55, 45, 29, 11,
        1, 17, 50, 58, 61, 45, 22, 53, 22, 21, 61, 15, 37, 58, 29,  7,  2,
       29,  8, 63, 55, 43, 23, 32, 60, 28, 56,  5, 38, 50, 54, 49])

In [35]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'do poison need,\nNor do I thee: though I did wish him dead,\nI hate the murderer, love him murdered.\nT'

Next Char Predictions:
 b"KazWvZ3i Q\nSyDnyvw c&v'iOSWWAhPkXu,e?KxDRwKbeX-qvRVpNrmnT.xUky!opfP:\nDksvfInIHvBXsP, P-xpdJSuOq&Ykoj"


#### Train Model

In [36]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [37]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.191004, shape=(), dtype=float32)


In [38]:
tf.exp(example_batch_mean_loss).numpy()

np.float32(66.089096)

In [39]:
model.compile(optimizer='adam', loss=loss)

In [40]:
import os

# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
os.makedirs(checkpoint_dir, exist_ok=True)

# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}.weights.h5")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True
)

In [41]:
EPOCHS = 20

In [38]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 212s 1s/step - loss: 3.0425
Epoch 2/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 208s 1s/step - loss: 1.9115
Epoch 3/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 193s 1s/step - loss: 1.6297
Epoch 4/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 192s 1s/step - loss: 1.4786
Epoch 5/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 192s 1s/step - loss: 1.3937
Epoch 6/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 195s 1s/step - loss: 1.3307
Epoch 7/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 193s 1s/step - loss: 1.2820
Epoch 8/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 192s 1s/step - loss: 1.2363
Epoch 9/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 191s 1s/step - loss: 1.1941
Epoch 10/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 193s 1s/step - loss: 1.1547
Epoch 11/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 195s 1s/step - loss: 1.1114
Epoch 12/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 192s 1s/step - loss: 1.0701
Epoch 13/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 191s 1s/step - loss: 1.0203
Epoch 14/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 191s 1s/step - loss: 0.9768
Epoch 15/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 19

#### Generate Teks

In [42]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [43]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [44]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

ROMEO:vB
.,
?uAfDdPXhJ??Sh,ZxRmk,QgicEQEhNzIe!
F.r,UvQzR,jy?AboX!$x:pDIqxTf;V
;rhc3Nh?b BKC,PU&-b.'Rp:HxLT?Rho?Tzt$zE;a?SOXAHJd.D.dDGbFoLWuww&ajCzOBxqQUY.gQw'cj'de?ABDxsrmtgV.rb.IlI$vjA.XmS&V!o-i-xmRzsGuhPb,
gEk.Zh;Es3?f-;;kuI'E&vwd,trHcK'R3QWsA'eq&c'XWq3z
wgHDjjvhr.,UneaJV:qGnkOd?Dcgh;l:tuA-UsrD3w.jpzeue?ZAWHZyoNwl,cvUfQ?mGvXZmfGV!!?L-;VOTkNImr.
GBR-foHv3wwedHxHU$j;IJULXznF D&JOFpaF ro!u
X,iWOt'dxNd
nr.UsM-3sHyO?-CMbidmAbqhJaasush
RthYVHhENNO
KlRF
 sqTo&sWN.?,rea-: !zFDjRqJzboKlNkQ-etP:Jin.gy;L
V epS!U,ad&fay:sa?,c;H!dQdmBVUmiy'jvsIkPcrScwcvVu?n'fhGpyzYV$mKC!
$;.YDaKirM cRmipy$oncNnSpXXvbMK'i;xKIkKs3SkAjxQPy.ZXUtYnMQzjjb?AmYS
Ic:y.lZ,ZIFotDA?BrCNMF?JSTF
cKlgGshKq3!cOzLl??hJkff3,vAooVP3$,m&d-b.Yhmg$x$n.,BMKHoMwjSTVbns&HfhUhgSTReLB!MNR.yQFAJDfUkVNmy3RpDpCLZdZNWtJLNeJjvx&wGztvi-qXN?qSHLH!,x vHa
yhjc-tCwtTiY:-;YRHRpX
RLofWr!CK
wJT3Tt$C!el VzPmB$Y&IUESjpY-u:XBWeOTk-RpCdzgV;DUz,dw3EF&XBjQjVXyhU-Cjn$QsREgYGGTRtEFVMmW O jq3XLXSt:pWo:CLRRcEw'J.cW'tdcDHb:;eiCQqnR,Wu$tdyjiZUIhmD&IgC-BiX&kpwVyzuP

In [45]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor(
[b"ROMEO:n\n:UD;l-qc\nt?;s-S,tqxH!&gJgs:wm  DKS.jzh&,m:\nRRzEwa$fUkBmo:rDRXWJYGk$v'NhvOLEY!NI GrR3T?luYYKKHZFCBZkTc?MDWsvFyFIsznXnGxwaV\nLIxcdCMtBF,'kRFDpvzMRUCRRKncdYbCKs!y.?Mr-ByCeAmsPBoRfbCVkQ3Iwij.vegqLXs Ccv:khB'R$qfeYG\nuFb$hE.CoQu$ooe-f;XUDUDQvdaHqYlFSFpES.gxqNgeI\ndb'jJ!VPUJSMOga'oHBhkhvn3'SgbovINF,:,RaerE,dDyWNTAHZc3EhfoDtF:osPks .KpYXTE.CugGDCqGjIksVx!g JAO-qDRiIPNndv&$PtLpp\nNM&ZGVeiUzjvDg,CGEfJXeFm.do:LvaIzyPasR,XkFIRrLqrJJ?qodhV&rN&sm:aUAcJD\nbCxR;zvNZu-vx&pIMP&: :RSzFhEpQOljQQ.\n;zIbTBQBHfmh?NAs,ALSNUNpW?n.rsKX pp!JjvtF,pEB&sl!gWAd:&3CKRBxa ldOKCBIk\nX:CU,W:elkytgbQ! sa,l?,PVbAyvvYXnh&QWJogdehdtOuKpU&n$NkgrjPniAb-TIiBFzW.AGDI'TNiq!DueEouPAn?XheqwRs$AS,gqf$OE&NJLjoV;D,STkffhgZD OMGlwHeWCALt,ND!URl?OIw;x!Vj3iey$kdTnYm-AORLYIJKD.,N!fAYoxtvJt\nKsPEYaBaHSllor\nfB;LqzthGHyJdmmQ.bY!UAHOFCUTp;b?ZbQ.dvbw-rxGR,uD:-&ukLxyt3RlANdJot-LnfaqCNhlmn:B,$\nV&aGQ.qQe\nluormbWGulEIVKE3,inWOK$.kUJ!rYq$Bxc.W$VOW!IsO\n!HWbiUQc!XeKPTPUnjjCXHp-bC?DGHc?ZGIDwlyu:sBgxDn:psY-Cfpx'S!&S!LO:xN

#### Ekspor Model Generator

In [46]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

INFO:tensorflow:Assets written to: one_step\assets


INFO:tensorflow:Assets written to: one_step\assets


In [47]:
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

ROMEO:ukp,GDh?ms; DS'FTFkkPwLhYS!3&
RdQbEfPcUL IANkp
NQZHibx!DjsYUTjnCmj:bspxJLChB:BThZiCMhLrST;h$CvDtpXM;


## TUGAS

In [2]:
import tensorflow as tf
import numpy as np
import os
import time

In [48]:
class CustomTraining(MyModel):
  @tf.function
  def train_step(self, inputs):
      inputs, labels = inputs
      with tf.GradientTape() as tape:
          predictions = self(inputs, training=True)
          loss = self.loss(labels, predictions)
      grads = tape.gradient(loss, model.trainable_variables)
      self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

      return {'loss': loss}

In [49]:
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [50]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [51]:
model.fit(dataset, epochs=1)

172/172 ━━━━━━━━━━━━━━━━━━━━ 208s 1s/step - loss: 2.4956


In [53]:
EPOCHS = 10

mean = tf.metrics.Mean()

for epoch in range(EPOCHS):
    start = time.time()

    # Memperbaiki kesalahan reset_state (tanpa "s")
    mean.reset_state()

    for (batch_n, (inp, target)) in enumerate(dataset):
        logs = model.train_step([inp, target])
        mean.update_state(logs['loss'])

        if batch_n % 50 == 0:
            template = f"Epoch {epoch+1} Batch {batch_n} Loss {logs['loss']:.4f}"
            print(template)

    # saving (checkpoint) the model every 5 epochs
    if (epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))

    print()
    print(f'Epoch {epoch+1} Loss: {mean.result().numpy():.4f}')
    print(f'Time taken for 1 epoch {time.time() - start:.2f} sec')
    print("_"*80)

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 2.0224
Epoch 1 Batch 50 Loss 1.9126
Epoch 1 Batch 100 Loss 1.7760
Epoch 1 Batch 150 Loss 1.7235

Epoch 1 Loss: 1.8319
Time taken for 1 epoch 196.50 sec
________________________________________________________________________________
Epoch 2 Batch 0 Loss 1.6676
Epoch 2 Batch 50 Loss 1.5974
Epoch 2 Batch 100 Loss 1.5855
Epoch 2 Batch 150 Loss 1.5435

Epoch 2 Loss: 1.5904
Time taken for 1 epoch 187.88 sec
________________________________________________________________________________
Epoch 3 Batch 0 Loss 1.4802
Epoch 3 Batch 50 Loss 1.4806
Epoch 3 Batch 100 Loss 1.4817
Epoch 3 Batch 150 Loss 1.4256

Epoch 3 Loss: 1.4629
Time taken for 1 epoch 198.85 sec
________________________________________________________________________________
Epoch 4 Batch 0 Loss 1.3679
Epoch 4 Batch 50 Loss 1.3887
Epoch 4 Batch 100 Loss 1.3252
Epoch 4 Batch 150 Loss 1.3540

Epoch 4 Loss: 1.3836
Time taken for 1 epoch 195.27 sec
_________________________________________________________________